In [1]:
# import modules
import os
import shutil
from osgeo import gdal
import pandas as pd 
pd.set_option('display.max_rows', 10)
import geopandas as gpd
from tqdm.notebook import tqdm

In [2]:
# track time
import time
start = time.time()
last = time.time()

## Generate area-of-interest for FUAs

In [3]:
# import csv containing lat/long for FUAs and create points gpd
fua_df = pd.read_csv('inputs/FUA-test.csv')
fua_gdf = gpd.GeoDataFrame(fua_df, geometry=gpd.points_from_xy(fua_df.long, fua_df.lat), crs='EPSG:4326').to_crs(2193)

# create 70km buffer to get aoi
fua_gdf['aoi'] = fua_gdf.buffer(70000)

print(fua_gdf)
print() 
print('buffering FUAs complete')
print(time.time() - last, 'seconds')
last = time.time()
print() 

    Name      long      lat                         geometry  \
0  Taupo  176.0704 -38.6843  POINT (1867076.976 5713780.848)   

                                                 aoi  
0  POLYGON ((1937076.976 5713780.848, 1936739.907...  

buffering FUAs complete
0.030347108840942383 seconds



##Remove undevelopable land parcels

In [4]:
# import LCDB and return water bodies 
water_bodies = gpd.read_file('inputs/lcdb-v50-land-cover-database-version-50-mainland-new-zealand.shp')

water_bodies = water_bodies[water_bodies.Name_2018.isin(['Not land', 'Lake or Pond', 'River', 'Estuarine Open Water', 'Mangrove']) == False]

print(water_bodies)
print()
print('loading water_bodies complete')
print(time.time() - last, 'seconds')
last = time.time()
print()

                               Name_2018                         Name_2012  \
0           Herbaceous Saline Vegetation      Herbaceous Saline Vegetation   
1           Herbaceous Saline Vegetation      Herbaceous Saline Vegetation   
3           Herbaceous Saline Vegetation      Herbaceous Saline Vegetation   
5           Herbaceous Saline Vegetation      Herbaceous Saline Vegetation   
6           Herbaceous Saline Vegetation      Herbaceous Saline Vegetation   
...                                  ...                               ...   
511099           Low Producing Grassland           Low Producing Grassland   
511100                 Indigenous Forest                 Indigenous Forest   
511101                     Exotic Forest                     Exotic Forest   
511102  Herbaceous Freshwater Vegetation  Herbaceous Freshwater Vegetation   
511103   High Producing Exotic Grassland   High Producing Exotic Grassland   

                               Name_2008                       

##Calculate slope for developable land

In [5]:
# filepath to dem data
dem = 'inputs/nz-dem-15m-2193.kea'

# create output dir for slope images and tmp dir for other files
slope_outputs = 'outputs/slope_images/'
tmp_dir = 'tmp/'
for folder in slope_outputs, tmp_dir:
    if not os.path.exists(folder):
            os.mkdir(folder)

# Clip water_bodies by each aoi
for index, row in tqdm(fua_gdf.iterrows(), desc='calc slope for FUAs'):
    # create tmp_dir for unrequired files and namePath for slope outputs
    tmpPath = tmp_dir + row['Name'] + '/'
    if not os.path.exists(tmpPath):
        os.mkdir(tmpPath)
    namePath = slope_outputs + row['Name']
    
    print(row['Name'])

    aoi_path = tmpPath + 'aoi.shp'
    developable_area = water_bodies.geometry.clip(row['aoi'])
    developable_area.to_file(aoi_path)

    developable_area_path = tmpPath + 'aoi.kea'

    # mask dem to get potential developable pixels within FUA aoi with gdal warp
    # output to memory
    gdal.Warp(destNameOrDestDS=developable_area_path,
    srcDSOrSrcDSTab=dem, format='kea', dstSRS= 'EPSG:2193', cutlineDSName=aoi_path, cutlineLayer='aoi', cropToCutline=True)

    print('masking dem complete')
    print(time.time() - last, 'seconds')
    last = time.time()
    print() 

    # Calculate slope using gdal DEMprocessing and warp to EPSG:2193
    destPath = namePath + '-slope.kea'
    #srcDS = developable_area_path

    gdal.DEMProcessing(destName=destPath, srcDS=developable_area_path, processing='slope', format='kea', slopeFormat='degree')
    
    # Remove tmp folder
    shutil.rmtree(tmpPath)
    
    print('calculating slope complete')
    print(time.time() - last, 'seconds')
    last = time.time()
    print()
    
shutil.rmtree(tmp_dir)

print('Total processing time')
print(time.time() - start, 'seconds')
last = time.time()
print()

calc slope for FUAs: 0it [00:00, ?it/s]

Name                                                    Taupo
long                                                 176.0704
lat                                                  -38.6843
geometry         POINT (1867076.9756865124 5713780.847506042)
aoi         POLYGON ((1937076.9756865124 5713780.847506042...
Name: 0, dtype: object


/Users/ben/miniconda3/envs/calc-developable-land/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
Warning 1: Caught exception in KEADataset constructor Could not retrieve image band meta data.


masking dem complete
90.65183115005493 seconds

calculating slope complete
8.951367855072021 seconds

Total processing time
139.09528017044067 seconds

